## Building Agent with Custom and Community Tools 🤖

<img src="./Images/Agents_Building.png" width="900" height="700" style="display: block; margin: auto;">

In [2]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load = load_dotenv('./../.env')


llm = ChatOllama(
    base_url="http://localhost:11434",
    model = "qwen2.5:latest",
    temperature=0.5,
    max_tokens = 250
)

## Bringing back the code from Previous section for Tool binding with LLM

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import tool

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

@tool
def add_numbers(a: int, b: int) -> int:
    "Add two numbers and return results."
    return int(a) + int(b)

@tool
def subtract_numbers(a: int, b: int) -> int:
    "Subtract two numbers and return results."
    return int(a) - int(b)

@tool
def multiply_numbers(a: int, b: int) -> int:
    "Multiply two numbers and return results."
    return int(a) * int(b)

tools = [wikipedia, add_numbers, subtract_numbers, multiply_numbers]


### Agent code 

In [4]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools= tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "What is the sum of 2 and 4. Did donald trump won the 2024 presidential election and became president in 2025?. Structure the response for me in JSON format"

result = agent.run(query)

print(result)




/var/folders/jc/c7p4f2sd36xbqwkp2djn8flc0000gn/T/ipykernel_91249/1976165713.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/var/folders/jc/c7p4f2sd36xbqwkp2djn8flc0000gn/T/ipykernel_91249/1976165713.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(que



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "wikipedia",
  "action_input": "Donald Trump 2024 presidential election"
}
```


Observation: Page: List of Donald Trump 2024 presidential campaign endorsements
Summary: A range of notable individuals, groups, and organizations endorsed Donald Trump for the 2024 U.S. presidential election.

Page: Donald Trump 2024 presidential campaign
Summary: Donald Trump, who served as the 45th president of the United States from 2017 to 2021, announced his campaign for the 2024 U.S. presidential election on November 15, 2022. After he won a landslide victory in the 2024 Iowa Republican presidential caucuses, Trump was generally described as being the Republican Party's presumptive nominee. He was officially nominated on July 15, 2024, at the Republican National Convention in Milwaukee, when he also announced JD Vance, a junior U.S. Senator from Ohio, as the nominee for vice president. The two initially faced off against the presumpti

In [12]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    
    ("system", "You are an expert in Math and latest news across the globe"),
    ("user", "Whats the sum of 2 and 5"),
    ("user", "What is the movie of Tom cruise hitting theater in 2025"),
    ("user", "give me both the answers in JSON Format")
    
])

result = agent.run(prompt_template)

print(result)



> Entering new AgentExecutor chain...
Thought: I need to provide the sum of 2 and 5, and find out a movie related to Tom Cruise hitting theaters in 2025. For the first part, I can use the add_numbers tool. However, for the second part, I will use Wikipedia to get information about a potential Tom Cruise movie.

Action:
```
{
  "action": "add_numbers",
  "action_input": {
    "a": 2,
    "b": 5
  }
}
```

Observation: 7
Thought:Now that I have the sum of 2 and 5, which is 7, I will use Wikipedia to find information about a potential Tom Cruise movie hitting theaters in 2025.
Action:
```
{
  "action": "wikipedia",
  "action_input": {
    "query": "Tom Cruise movies 2025"
  }
}
```
Observation: Page: Tom Cruise filmography
Summary: Tom Cruise is an American actor and producer who made his film debut with a minor role in the 1981 romantic drama Endless Love. Two years later, he made his breakthrough by starring in the romantic comedy Risky Business (1983), which garnered his first nomina

### Using Playwright Browser Tool

In [13]:
!pip install --upgrade --quiet  playwright


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser
)
import nest_asyncio

nest_asyncio.apply()


In [6]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-p

In [7]:
tools_by_name = {tool.name: tool for tool in tools}
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]
navigate_tool, get_elements_tool

(NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>),
 GetElementsTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/karthik/Library/Caches/ms-playwright/chromium-1155/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=133.0.6943.16>))

In [23]:
await navigate_tool.arun({"url": "http://eaapp.somee.com/Employee"})

'Navigating to http://eaapp.somee.com/Employee returned status code 200'

In [24]:
await get_elements_tool.arun({"selector": "td", "attributes": ["innerText"]})

'[{"innerText": "Karthik"}, {"innerText": "4000"}, {"innerText": "24"}, {"innerText": "1"}, {"innerText": "karthik@executeautomation.com"}, {"innerText": "Benefits Edit Delete"}, {"innerText": "Prashanth"}, {"innerText": "7000"}, {"innerText": "30"}, {"innerText": "2"}, {"innerText": "prashanth@executeautomation.com"}, {"innerText": "Benefits Edit Delete"}, {"innerText": "Ramesh"}, {"innerText": "3500"}, {"innerText": "13"}, {"innerText": "2"}, {"innerText": "ramesh@executeautomation.com"}, {"innerText": "Benefits Edit Delete"}, {"innerText": "John"}, {"innerText": "2500"}, {"innerText": "18"}, {"innerText": "3"}, {"innerText": "john@executeautomation.com"}, {"innerText": "Benefits Edit Delete"}, {"innerText": "Selena"}, {"innerText": "200"}, {"innerText": "3"}, {"innerText": "3"}, {"innerText": "Selena@example.com"}, {"innerText": "Benefits Edit Delete"}, {"innerText": "Selena"}, {"innerText": "200"}, {"innerText": "3"}, {"innerText": "3"}, {"innerText": "Selena@example.com"}, {"inner

### Agent to invoke the Playwright Browser Tool

In [9]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools= tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "Extract the table data from http://eaapp.somee.com/Employee page and get the salary average for all the employees"

result = await agent.arun(query)

print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "http://eaapp.somee.com/Employee"
  }
}
```

Observation: Navigating to http://eaapp.somee.com/Employee returned status code 200
Thought:Action:
```
{
  "action": "get_elements",
  "action_input": {
    "selector": "table tr td:nth-child(3)",
    "attributes": []
  }
}
``` 

Observation: []
Thought:It seems that the table data for salaries is not directly accessible using a simple CSS selector. We need to inspect the page structure more carefully or use JavaScript execution if necessary.

Action:
```
{
  "action": "get_elements",
  "action_input": {
    "selector": "table tr td:nth-child(3)",
    "attributes": []
  }
}
``` 

Observation: []
Thought:It seems that the data might be loaded dynamically or is not directly accessible via a simple CSS selector. Let's try to extract all table rows and then calculate the average salary from there.

Action:
```
{
  "action": "get_

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: To calculate the average salary of all employees, we need to extract the salary values from the text and then compute their average. Here's how we can proceed:

### Step-by-Step Plan:
1. **Extract Salary Values**: Identify and extract all the salary values from the provided text.
2. **Convert to Numbers**: Convert these extracted strings into numerical values.
3. **Calculate Average Salary**: Compute the average of these numbers.

Let's go through this step-by-step using Python code for clarity:

### Step 1: Extract Salary Values
From the given text, we can see that salaries are in the third column (assuming the structure is consistent).

```python
import re

# Provided text
text = """
Execute Automation Employee App Home About Employee List Register Login Name Salary Dur